# Problem Set 2

## Preparation

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pytz

###Import data

In [52]:
from google.colab import drive
drive.mount("/content/drive")
!mkdir -p datashim
!cp /content/drive/Shareddrives/DS602-S22/Data/data602.py ./datashim
from datashim import data602

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:

cdf = data602.read_citations('/content/drive/Shareddrives/DS602-S22/Data/Parking_and_Moving_Citations.csv')
wdf = data602.read_weather('/content/drive/Shareddrives/DS602-S22/Data/weather_clean.csv')

## Question 1

Using the `weather` dataset, construct a derived data frame having the following qualities:

1. Each row represents an observation day
2. Each column represents an hourly temperature observation.  That is, the dataframe includes 24 columns of the form `tmpmeasx`	where x ranges from 0 to 23 and represents the observation taken that hour.  (For changes from daylight savings time to standard time, there are two observations at the 1:00 hour (local).  You may discard one of the values arbitrarily.)

In [54]:
wdf

,date,wnd_speed,tmp_meas,precip,precip_intensity,cloudcover,tstorm,winter,mist,fog
0,2015-12-31 19:54:00-05:00,1.5,7.2,0.0,0,4.0,0,0,0,0
1,2015-12-31 20:54:00-05:00,3.1,6.1,0.0,0,4.0,0,0,0,0
2,2015-12-31 21:54:00-05:00,2.1,6.7,0.0,0,4.0,0,0,0,0
3,2015-12-31 22:54:00-05:00,2.6,6.1,0.0,0,4.0,0,0,0,0
4,2015-12-31 23:54:00-05:00,2.1,5.6,0.0,0,4.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
48554,2021-07-16 15:54:00-04:00,4.6,36.7,0.0,0,2.0,0,0,0,0
48555,2021-07-16 16:54:00-04:00,4.1,35.6,0.0,0,2.0,0,0,0,0
48556,2021-07-16 17:54:00-04:00,4.1,33.3,0.0,0,3.0,0,0,0,0
48557,2021-07-16 18:54:00-04:00,3.6,32.8,0.0,0,2.0,0,0,0,0


In [55]:
wdf['day'] = wdf['date'].dt.date
wdf

,date,wnd_speed,tmp_meas,precip,precip_intensity,cloudcover,tstorm,winter,mist,fog,day
0,2015-12-31 19:54:00-05:00,1.5,7.2,0.0,0,4.0,0,0,0,0,2015-12-31
1,2015-12-31 20:54:00-05:00,3.1,6.1,0.0,0,4.0,0,0,0,0,2015-12-31
2,2015-12-31 21:54:00-05:00,2.1,6.7,0.0,0,4.0,0,0,0,0,2015-12-31
3,2015-12-31 22:54:00-05:00,2.6,6.1,0.0,0,4.0,0,0,0,0,2015-12-31
4,2015-12-31 23:54:00-05:00,2.1,5.6,0.0,0,4.0,0,0,0,0,2015-12-31
...,...,...,...,...,...,...,...,...,...,...,...
48554,2021-07-16 15:54:00-04:00,4.6,36.7,0.0,0,2.0,0,0,0,0,2021-07-16
48555,2021-07-16 16:54:00-04:00,4.1,35.6,0.0,0,2.0,0,0,0,0,2021-07-16
48556,2021-07-16 17:54:00-04:00,4.1,33.3,0.0,0,3.0,0,0,0,0,2021-07-16
48557,2021-07-16 18:54:00-04:00,3.6,32.8,0.0,0,2.0,0,0,0,0,2021-07-16


In [56]:
derived_df=pd.DataFrame(np.NaN,columns=["tmpmeas"+str(i) for i in range(0,24)],index=wdf.groupby([pd.DatetimeIndex(wdf['date']).date])['tmp_meas'].count().index)
count=0
dates = pd.DatetimeIndex(wdf['date']).date
for value in wdf.values:
  column = "tmpmeas"+str(value[0].hour)
  derived_df.loc[dates[count]].at[column]=value[2]
  count+=1

In [57]:
derived_df.head()

,tmpmeas0,tmpmeas1,tmpmeas2,tmpmeas3,tmpmeas4,tmpmeas5,tmpmeas6,tmpmeas7,tmpmeas8,tmpmeas9,tmpmeas10,tmpmeas11,tmpmeas12,tmpmeas13,tmpmeas14,tmpmeas15,tmpmeas16,tmpmeas17,tmpmeas18,tmpmeas19,tmpmeas20,tmpmeas21,tmpmeas22,tmpmeas23
2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.2,6.1,6.7,6.1,5.6
2016-01-01,5.0,4.4,3.9,3.3,3.3,4.4,3.9,3.9,4.4,5.6,5.6,6.7,6.7,6.1,5.6,5.0,4.4,3.9,2.2,1.1,0.6,-0.6,0.6,0.6
2016-01-02,-0.6,-1.7,-1.7,-0.6,-2.8,-2.2,-2.8,-2.2,1.1,3.3,5.0,6.1,7.2,8.3,8.3,7.8,6.1,3.9,2.2,1.7,0.6,0.0,-1.1,-1.7
2016-01-03,-1.7,-1.7,-2.2,-2.8,-2.8,-0.6,-1.1,-2.8,1.7,5.6,7.8,9.4,10.0,10.0,11.1,10.6,8.9,5.6,3.9,2.8,1.7,0.6,0.0,-1.1
2016-01-04,-2.2,-1.7,-2.8,-1.7,0.0,0.0,0.6,0.6,0.6,0.0,1.1,2.2,2.2,1.7,1.1,0.0,-1.7,-2.2,-2.8,-3.3,-5.0,-6.1,-7.2,-8.3


Using this derived dataset, show the average difference in temperature, standard deviation, and max/min values between observations taken in hours 2 (about 2:56am) and 14 (about 14:56, or 2:56pm).

In [58]:
mean=derived_df['tmpmeas14'].mean()-derived_df['tmpmeas2'].mean()
print('The Average Difference in temperature is:',mean)
sd=derived_df['tmpmeas14'].std()-derived_df['tmpmeas2'].std()
print('The Difference in Standard deviation is:',sd)
max_dif=derived_df['tmpmeas14'].max()-derived_df['tmpmeas2'].max()
print('The Maximun difference value between the 2 hours is:',max_dif)
min_dif=derived_df['tmpmeas14'].min()-derived_df['tmpmeas2'].min()
print('The Minimum difference value between the 2 hours is:',min_dif)

The Average Difference in temperature is: 7.031541276730765
The Difference in Standard deviation is: 0.9653120779038371
The Maximun difference value between the 2 hours is: 7.800000000000004
The Minimum difference value between the 2 hours is: 5.6


## Question 2

Circular data and One-Hot Encoding} Open and clean the `citations` data frame using the script provided, and discard records where the violation timestamp is not between January 1, 2020 and July 1, 2021.


In [59]:
cdf

,citation,tag,state,violcode,description,violfine,violdate,location,violdtl
0,91,03W671,MD,27,No Stop/Park Street Cleaning,52,2011-12-09 11:55:00-05:00,<NA>,2011-12-09 11:55:00
1,166,T294093T,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2011-12-09 12:39:00-05:00,<NA>,2011-12-09 12:39:00
2,299,19TLR318,ME,99,All Other Stopping or Parking Violations,32,2011-12-09 11:55:00-05:00,<NA>,2011-12-09 11:55:00
3,315,14TLR551,ME,99,All Other Stopping or Parking Violations,32,2011-12-09 12:02:00-05:00,<NA>,2011-12-09 12:02:00
4,562,9AE2340,MD,18,All Other Parking Meter Violations,32,2011-12-12 10:47:00-05:00,LAFAYETTE MARKET LOT Baltimore MD,2011-12-12 10:47:00
...,...,...,...,...,...,...,...,...,...
5421701,99999956,5BN6816,MD,18,All Other Parking Meter Violations,32,2017-05-08 16:06:00-04:00,1000 CHARLES ST Baltimore MD,2017-05-08 16:06:00
5421702,99999964,9CT1108,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2017-05-08 16:44:00-04:00,1100 LIGHT ST Baltimore MD,2017-05-08 16:44:00
5421703,99999972,9CG6913,MD,8,No Stopping/Standing Tow Away Zone,52,2017-05-08 16:18:00-04:00,2400 HILTON ST Baltimore MD,2017-05-08 16:18:00
5421704,99999980,A300187,MD,18,All Other Parking Meter Violations,32,2017-05-08 16:56:00-04:00,800 BALTIMORE ST Baltimore MD,2017-05-08 16:56:00


In [60]:
derived_cdf=cdf[(cdf['violdate']>'2020-01-01 00:00:00') & (cdf['violdate']<'2021-07-01 00:00:00')]
derived_cdf

,citation,tag,state,violcode,description,violfine,violdate,location,violdtl
2627,130278,19213CD,MD,27,No Stop/Park Street Cleaning,52,2021-06-06 23:40:00-04:00,"(39.28997272, -76.61002178)",2021-06-06 23:40:00
2632,130344,9EL0846,MD,3,Obstruct/Impeding Flow of Traffic,102,2021-04-03 02:59:00-04:00,<NA>,2021-04-03 02:59:00
2633,130393,2EG0053,MD,3,Obstruct/Impeding Flow of Traffic,102,2021-04-11 01:30:00-04:00,<NA>,2021-04-11 01:30:00
2634,130427,13381CK,MD,3,Obstruct/Impeding Flow of Traffic,102,2021-04-11 01:35:00-04:00,<NA>,2021-04-11 01:35:00
2635,130435,1EC9345,MD,3,Obstruct/Impeding Flow of Traffic,102,2021-04-11 01:36:00-04:00,<NA>,2021-04-11 01:36:00
...,...,...,...,...,...,...,...,...,...
3444247,78829488,6CR1677,MD,22,Expired Tags,32,2021-01-27 13:30:00-05:00,<NA>,2021-01-27 13:30:00
3444248,78829496,751026,MD,8,No Stopping/Standing Tow Away Zone,52,2021-01-27 13:39:00-05:00,<NA>,2021-01-27 13:39:00
3452403,79827523,HIPY83,FL,8,No Stopping/Standing Tow Away Zone,52,2020-07-31 12:00:00-04:00,<NA>,2020-07-31 12:00:00
3452404,79827531,9DH4142,MD,99,All Other Stopping or Parking Violations,32,2021-03-16 08:00:00-04:00,<NA>,2021-03-16 08:00:00


Identify the 5 most common violation types.  Remove from the derivative data frame any violation types not in these 5 most common

In [61]:
result=derived_cdf.groupby(["violcode"])['citation'].count().reset_index()
c_viol=result.loc[result['citation']==max(result['citation'])].reset_index()['violcode'].values
print('Most Common violation type is :',c_viol)

Most Common violation type is : [32]


In [62]:
top_5 = derived_cdf['violcode'].value_counts()
print("5 most common violations are:\n",top_5.head(5))

5 most common violations are:
 32    847973
30    184821
31    164314
18    104007
12     53680
Name: violcode, dtype: int64


In [64]:
df1 = pd.DataFrame(data=top_5.index, columns=['violcode'])
df2 = pd.DataFrame(data=top_5.values, columns=['value'])
df = pd.merge(df1, df2, left_index=True, right_index=True)
df.head()

,violcode,value
0,32,847973
1,30,184821
2,31,164314
3,18,104007
4,12,53680


In [63]:
drop_df = derived_cdf[ (derived_cdf['violcode'] != 32) & (derived_cdf['violcode'] != 30) & (derived_cdf['violcode'] != 31) & (derived_cdf['violcode'] != 18) & (derived_cdf['violcode'] != 12)].index
derived_cdf.drop(drop_df , inplace=True)
derived_cdf

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,citation,tag,state,violcode,description,violfine,violdate,location,violdtl
2640,130484,3DM4068,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-06-04 01:35:00-04:00,"(39.28865729, -76.61323964)",2021-06-04 01:35:00
7318,515841,7DD3863,MD,18,All Other Parking Meter Violations,32,2020-05-27 11:15:00-04:00,<NA>,2020-05-27 11:15:00
8776,604827,1ED9532,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-03-10 09:36:00-05:00,<NA>,2021-03-10 09:36:00
8818,608240,2CB7909,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2020-11-22 15:50:00-05:00,<NA>,2020-11-22 15:50:00
18361,1213206,9AT4802,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-04-16 07:55:00-04:00,<NA>,2021-04-16 07:55:00
...,...,...,...,...,...,...,...,...,...
3387562,69999515,1EL2879,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-06-16 11:23:00-04:00,"(39.29417728, -76.61859607)",2021-06-16 11:23:00
3387563,69999523,72533CJ,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-06-16 11:26:00-04:00,"(39.29417728, -76.61859607)",2021-06-16 11:26:00
3387564,69999531,5DL3983,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-06-16 11:28:00-04:00,"(39.29417728, -76.61859607)",2021-06-16 11:28:00
3387565,69999549,3BV4229,MD,18,All Other Parking Meter Violations,32,2021-06-16 11:34:00-04:00,"(39.29417728, -76.61859607)",2021-06-16 11:34:00


Identify the average time of day for each violation in the derivative data frame.  (You must use the circular mean to compute this average.)

In [77]:
derived_cdf['day'] = derived_cdf['violdtl'].dt.day
derived_cdf['month'] = derived_cdf['violdtl'].dt.month
derived_cdf['year'] = derived_cdf['violdtl'].dt.year
derived_cdf['time']= derived_cdf['violdtl'].dt.hour
derived_cdf


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,citation,tag,state,violcode,description,violfine,violdate,location,violdtl,day,month,year,time
2640,130484,3DM4068,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-06-04 01:35:00-04:00,"(39.28865729, -76.61323964)",2021-06-04 01:35:00,4,6,2021,1
7318,515841,7DD3863,MD,18,All Other Parking Meter Violations,32,2020-05-27 11:15:00-04:00,<NA>,2020-05-27 11:15:00,27,5,2020,11
8776,604827,1ED9532,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-03-10 09:36:00-05:00,<NA>,2021-03-10 09:36:00,10,3,2021,9
8818,608240,2CB7909,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2020-11-22 15:50:00-05:00,<NA>,2020-11-22 15:50:00,22,11,2020,15
18361,1213206,9AT4802,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-04-16 07:55:00-04:00,<NA>,2021-04-16 07:55:00,16,4,2021,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387562,69999515,1EL2879,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-06-16 11:23:00-04:00,"(39.29417728, -76.61859607)",2021-06-16 11:23:00,16,6,2021,11
3387563,69999523,72533CJ,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-06-16 11:26:00-04:00,"(39.29417728, -76.61859607)",2021-06-16 11:26:00,16,6,2021,11
3387564,69999531,5DL3983,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2021-06-16 11:28:00-04:00,"(39.29417728, -76.61859607)",2021-06-16 11:28:00,16,6,2021,11
3387565,69999549,3BV4229,MD,18,All Other Parking Meter Violations,32,2021-06-16 11:34:00-04:00,"(39.29417728, -76.61859607)",2021-06-16 11:34:00,16,6,2021,11


In [80]:
mean1=derived_cdf['time'].mean()
mean1

13.003693547732313

The violation type column is a nominal field.  Convert this field to one-hot encoded (OHE) variables, and add these variables to the data frame.

In [65]:
x=pd.get_dummies(derived_cdf['violcode'], dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)
x

,12,18,30,31,32
2640,1,0,0,0,0
7318,0,1,0,0,0
8776,1,0,0,0,0
8818,1,0,0,0,0
18361,1,0,0,0,0
...,...,...,...,...,...
3387562,1,0,0,0,0
3387563,1,0,0,0,0
3387564,1,0,0,0,0
3387565,0,1,0,0,0


In [73]:
result=pd.concat([derived_cdf,x])
result

,12,18,30,31,32,citation,description,location,state,tag,violcode,violdate,violdtl,violfine
2640,NaN,NaN,NaN,NaN,NaN,130484.0,No Stopping/Standing Not Tow-Away Zone,"(39.28865729, -76.61323964)",MD,3DM4068,12.0,2021-06-04 01:35:00-04:00,2021-06-04 01:35:00,32.0
7318,NaN,NaN,NaN,NaN,NaN,515841.0,All Other Parking Meter Violations,<NA>,MD,7DD3863,18.0,2020-05-27 11:15:00-04:00,2020-05-27 11:15:00,32.0
8776,NaN,NaN,NaN,NaN,NaN,604827.0,No Stopping/Standing Not Tow-Away Zone,<NA>,MD,1ED9532,12.0,2021-03-10 09:36:00-05:00,2021-03-10 09:36:00,32.0
8818,NaN,NaN,NaN,NaN,NaN,608240.0,No Stopping/Standing Not Tow-Away Zone,<NA>,MD,2CB7909,12.0,2020-11-22 15:50:00-05:00,2020-11-22 15:50:00,32.0
18361,NaN,NaN,NaN,NaN,NaN,1213206.0,No Stopping/Standing Not Tow-Away Zone,<NA>,MD,9AT4802,12.0,2021-04-16 07:55:00-04:00,2021-04-16 07:55:00,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387562,1.0,0.0,0.0,0.0,0.0,NaN,NaN,<NA>,NaN,<NA>,NaN,NaT,NaT,NaN
3387563,1.0,0.0,0.0,0.0,0.0,NaN,NaN,<NA>,NaN,<NA>,NaN,NaT,NaT,NaN
3387564,1.0,0.0,0.0,0.0,0.0,NaN,NaN,<NA>,NaN,<NA>,NaN,NaT,NaT,NaN
3387565,0.0,1.0,0.0,0.0,0.0,NaN,NaN,<NA>,NaN,<NA>,NaN,NaT,NaT,NaN


## Question 3
 Join the citations and weather datasets to produce a merged data frame.  The merged data frame should provide, for each citation, the reported weather conditions for the closest weather observation.

In [66]:
wdf=wdf.sort_values("day")
cdf = cdf.sort_values("violdate")
merged_df = pd.merge(cdf, wdf, left_on="violdate", right_on="date")
merged_df

,citation,tag,state,violcode,description,violfine,violdate,location,violdtl,date,wnd_speed,tmp_meas,precip,precip_intensity,cloudcover,tstorm,winter,mist,fog,day
0,95413739,8BE9923,MD,8,No Stopping/Standing Tow Away Zone,52,2015-12-31 19:54:00-05:00,200 GITTINGS ST Baltimore MD,2015-12-31 19:54:00,2015-12-31 19:54:00-05:00,1.5,7.2,0.0,0,4.0,0,0,0,0,2015-12-31
1,95581402,1BW7473,MD,18,All Other Parking Meter Violations,32,2016-01-02 08:54:00-05:00,1300 ST PAUL ST Baltimore MD,2016-01-02 08:54:00,2016-01-02 08:54:00-05:00,0.0,1.1,0.0,0,0.0,0,0,0,0,2016-01-02
2,95581196,5AR3365,MD,12,No Stopping/Standing Not Tow-Away Zone,32,2016-01-02 16:54:00-05:00,100 22ND ST Baltimore MD,2016-01-02 16:54:00,2016-01-02 16:54:00-05:00,0.0,6.1,0.0,0,0.0,0,0,0,0,2016-01-02
3,95608338,2CE0758,MD,18,All Other Parking Meter Violations,32,2016-01-02 16:54:00-05:00,600 CENTRAL AVE Baltimore MD,2016-01-02 16:54:00,2016-01-02 16:54:00-05:00,0.0,6.1,0.0,0,0.0,0,0,0,0,2016-01-02
4,95615499,6AR4493,MD,18,All Other Parking Meter Violations,32,2016-01-02 17:54:00-05:00,U/B HOLLIDAY ST Baltimore MD,2016-01-02 17:54:00,2016-01-02 17:54:00-05:00,1.5,3.9,0.0,0,0.0,0,0,0,0,2016-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72247,21296448,3EM4765,MD,11,Residential Parking Permit Only,52,2021-07-15 12:54:00-04:00,<NA>,2021-07-15 12:54:00,2021-07-15 12:54:00-04:00,0.0,33.3,0.0,0,4.0,0,0,0,0,2021-07-15
72248,21349882,8CX7023,MD,18,All Other Parking Meter Violations,32,2021-07-15 13:54:00-04:00,<NA>,2021-07-15 13:54:00,2021-07-15 13:54:00-04:00,0.0,33.9,0.0,0,4.0,0,0,0,0,2021-07-15
72249,21340345,3EG3388,MD,18,All Other Parking Meter Violations,32,2021-07-15 13:54:00-04:00,<NA>,2021-07-15 13:54:00,2021-07-15 13:54:00-04:00,0.0,33.9,0.0,0,4.0,0,0,0,0,2021-07-15
72250,21312740,PBX7265,NC,17,Less Than 15 feet from Fire Hydrant,77,2021-07-15 17:54:00-04:00,<NA>,2021-07-15 17:54:00,2021-07-15 17:54:00-04:00,3.6,32.2,0.0,0,3.0,0,0,0,0,2021-07-15


From this joined data set, examine instances of fixed speed camera citations (violation type 32).  Does weather affect citation volumes?  Justify your conclusion. [Note: you do not need to provide a statistical proof; it is sufficient to justify your conclusion through visual or descriptive analysis.]

In [67]:
new_df=merged_df[merged_df["violcode"]==32]
new_df

,citation,tag,state,violcode,description,violfine,violdate,location,violdtl,date,wnd_speed,tmp_meas,precip,precip_intensity,cloudcover,tstorm,winter,mist,fog,day
8334,80000640,9AL8269,MD,32,Fixed Speed Camera,40,2017-07-31 06:54:00-04:00,<NA>,2017-07-31 06:54:00,2017-07-31 06:54:00-04:00,0.0,16.1,0.0,0,1.0,0,0,0,0,2017-07-31
8335,80000657,4CE0725,MD,32,Fixed Speed Camera,40,2017-07-31 06:54:00-04:00,1200 BLK SOUTH CATON AVE NB Baltimore MD,2017-07-31 06:54:00,2017-07-31 06:54:00-04:00,0.0,16.1,0.0,0,1.0,0,0,0,0,2017-07-31
8336,80000665,1CP9778,MD,32,Fixed Speed Camera,40,2017-07-31 06:54:00-04:00,1200 BLK SOUTH CATON AVE NB Baltimore MD,2017-07-31 06:54:00,2017-07-31 06:54:00-04:00,0.0,16.1,0.0,0,1.0,0,0,0,0,2017-07-31
8338,80001531,1CR8859,MD,32,Fixed Speed Camera,40,2017-07-31 07:54:00-04:00,2700 BLK GWYNNS FALLS PKY WB Baltimore MD,2017-07-31 07:54:00,2017-07-31 07:54:00-04:00,0.0,20.0,0.0,0,2.0,0,0,0,0,2017-07-31
8339,80001549,68818CF,MD,32,Fixed Speed Camera,40,2017-07-31 07:54:00-04:00,2700 BLK GWYNNS FALLS PKY WB Baltimore MD,2017-07-31 07:54:00,2017-07-31 07:54:00-04:00,0.0,20.0,0.0,0,2.0,0,0,0,0,2017-07-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72116,39505244,6CP2015,MD,32,Fixed Speed Camera,40,2021-07-05 10:54:00-04:00,<NA>,2021-07-05 10:54:00,2021-07-05 10:54:00-04:00,2.6,29.4,0.0,0,3.0,0,0,0,0,2021-07-05
72117,39505251,34864CG,MD,32,Fixed Speed Camera,40,2021-07-05 10:54:00-04:00,<NA>,2021-07-05 10:54:00,2021-07-05 10:54:00-04:00,2.6,29.4,0.0,0,3.0,0,0,0,0,2021-07-05
72118,39505269,4CJ2158,MD,32,Fixed Speed Camera,40,2021-07-05 10:54:00-04:00,<NA>,2021-07-05 10:54:00,2021-07-05 10:54:00-04:00,2.6,29.4,0.0,0,3.0,0,0,0,0,2021-07-05
72119,39506820,8EN0394,MD,32,Fixed Speed Camera,40,2021-07-05 11:54:00-04:00,<NA>,2021-07-05 11:54:00,2021-07-05 11:54:00-04:00,3.1,30.6,0.0,0,3.0,0,0,0,0,2021-07-05


In [68]:
weather_df= new_df[new_df["wnd_speed"]>=0.0]
weather_df

,citation,tag,state,violcode,description,violfine,violdate,location,violdtl,date,wnd_speed,tmp_meas,precip,precip_intensity,cloudcover,tstorm,winter,mist,fog,day
8334,80000640,9AL8269,MD,32,Fixed Speed Camera,40,2017-07-31 06:54:00-04:00,<NA>,2017-07-31 06:54:00,2017-07-31 06:54:00-04:00,0.0,16.1,0.0,0,1.0,0,0,0,0,2017-07-31
8335,80000657,4CE0725,MD,32,Fixed Speed Camera,40,2017-07-31 06:54:00-04:00,1200 BLK SOUTH CATON AVE NB Baltimore MD,2017-07-31 06:54:00,2017-07-31 06:54:00-04:00,0.0,16.1,0.0,0,1.0,0,0,0,0,2017-07-31
8336,80000665,1CP9778,MD,32,Fixed Speed Camera,40,2017-07-31 06:54:00-04:00,1200 BLK SOUTH CATON AVE NB Baltimore MD,2017-07-31 06:54:00,2017-07-31 06:54:00-04:00,0.0,16.1,0.0,0,1.0,0,0,0,0,2017-07-31
8338,80001531,1CR8859,MD,32,Fixed Speed Camera,40,2017-07-31 07:54:00-04:00,2700 BLK GWYNNS FALLS PKY WB Baltimore MD,2017-07-31 07:54:00,2017-07-31 07:54:00-04:00,0.0,20.0,0.0,0,2.0,0,0,0,0,2017-07-31
8339,80001549,68818CF,MD,32,Fixed Speed Camera,40,2017-07-31 07:54:00-04:00,2700 BLK GWYNNS FALLS PKY WB Baltimore MD,2017-07-31 07:54:00,2017-07-31 07:54:00-04:00,0.0,20.0,0.0,0,2.0,0,0,0,0,2017-07-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72116,39505244,6CP2015,MD,32,Fixed Speed Camera,40,2021-07-05 10:54:00-04:00,<NA>,2021-07-05 10:54:00,2021-07-05 10:54:00-04:00,2.6,29.4,0.0,0,3.0,0,0,0,0,2021-07-05
72117,39505251,34864CG,MD,32,Fixed Speed Camera,40,2021-07-05 10:54:00-04:00,<NA>,2021-07-05 10:54:00,2021-07-05 10:54:00-04:00,2.6,29.4,0.0,0,3.0,0,0,0,0,2021-07-05
72118,39505269,4CJ2158,MD,32,Fixed Speed Camera,40,2021-07-05 10:54:00-04:00,<NA>,2021-07-05 10:54:00,2021-07-05 10:54:00-04:00,2.6,29.4,0.0,0,3.0,0,0,0,0,2021-07-05
72119,39506820,8EN0394,MD,32,Fixed Speed Camera,40,2021-07-05 11:54:00-04:00,<NA>,2021-07-05 11:54:00,2021-07-05 11:54:00-04:00,3.1,30.6,0.0,0,3.0,0,0,0,0,2021-07-05


In [ ]:
weather_df.plot(x='violcode',y='wnd_speed',kind='bar',figsize=(20,15))
plt.show()

## From the above observation we can say that weather affects citation. We can see that for all the weather conditions like wnd_speed,tmp_meas,precip,precip_intensity,cloudcover,tstorm,winter,mist,fog greater than 0 the citations are reduced during harsh weather conditions.